In [7]:
import pandas as pd


df = pd.read_csv('./archive/2023_GradesVsPeriodAttendance_01.csv')

print(df.school_name.unique())

['West High School' 'North High School' 'South High School'
 'Hope High School' 'Sunny High School' 'Small High School'
 'East High School']


1. Vaikuttaako poissaolojen määrä oppilaan saamiin arvosanoihin?
- Kysymys: Onko oppilailla, joilla on enemmän poissaoloja, keskimäärin huonommat arvosanat?
- Analyysi: Tarkastelkaa poissaolojen määrää eri oppitunneilla ja verratkaa sitä oppilaan saamiin arvosanoihin.

In [ ]:
##Tähän koodisoluun voit kirjoittaa koodia, joka vastaa yllä esitettyyn kysymykseen.

2. Mitkä oppitunnit ovat herkimmin altistettuja poissaoloille?
-  Kysymys: Onko joillakin oppitunneilla (esim. ensimmäinen tai kuudes tunti) enemmän poissaoloja kuin muilla?
-  Analyysi: Vertailkaa poissaolomääriä eri oppitunneilla ja tutkikaa, onko tiettyjä jaksoja, joissa poissaoloja esiintyy enemmän.


In [ ]:
##Tähän koodisoluun voit kirjoittaa koodia, joka vastaa yllä esitettyyn kysymykseen.


3. Onko tietyn vuoden opiskelijoilla (esim. 9., 10., 11. tai 12. luokka) korkeampi poissaolojen määrä tai heikommat arvosanat?
-  Kysymys: Onko poissaolojen määrä ja oppilaan saamat arvosanat yhteydessä opiskelijan luokka-asteeseen?
-  Analyysi: Ryhmitelkää oppilaat heidän luokka-asteensa mukaan ja tarkastelkaa poissaolojen ja arvosanojen keskiarvoa kullekin luokka-asteelle.

In [ ]:
##Tähän koodisoluun voit kirjoittaa koodia, joka vastaa yllä esitettyyn kysymykseen.


4. Kuinka paljon poissaoloja tarvitaan, ennen kuin ne alkavat vaikuttaa merkittävästi oppilaan arvosanoihin?
-  Kysymys: Mikä on poissaolojen kynnysarvo, jonka jälkeen oppilaan arvosanat alkavat heiketä merkittävästi?
-  Analyysi: Mallintakaa poissaolojen ja arvosanojen välistä suhdetta ja selvittäkää, missä kohtaa poissaolot alkavat heikentää arvosanoja.

In [ ]:
##Tähän koodisoluun voit kirjoittaa koodia, joka vastaa yllä esitettyyn kysymykseen.

5. Voidaanko ennustaa opiskelijan menestys toisen jakson aikana ensimmäisen jakson perusteella?
-  Kysymys: Voidaanko ensimmäisen jakson arvosanojen ja poissaolojen perusteella ennustaa toisen jakson suoriutumista?
-  Analyysi: Käyttäkää ensimmäisen jakson tietoja mallina, ja testatkaa, kuinka hyvin ne ennustavat toisen jakson arvosanoja ja poissaoloja.


In [ ]:
##Tähän koodisoluun voit kirjoittaa koodia, joka vastaa yllä esitettyyn kysymykseen.